In [1]:
import weka.core.jvm as jvm
import os
import traceback
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.core.dataset import Instances
from weka.timeseries import TSEvaluation, TSEvalModule, WekaForecaster,Periodicity
from weka.classifiers import Classifier
from weka.core.classes import serialization_write, serialization_read
jvm.start(packages=True)

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\core.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\mtj.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'c:\\Users\\nptu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxH

In [2]:
# load a dataset
from weka.core.converters import Loader
loader = Loader(classname="weka.core.converters.ArffLoader")
airline_data = loader.load_file("data/BTC-2y.arff")
# airline_data.class_is_last()

In [3]:
# available evaluation modules
print("Evaluation modules")
modules = TSEvalModule.module_list()
print("Available modules")
for module in modules:
    print("-" + str(module))
print("Loading module by name")
print(TSEvalModule.module("MAE"))

Evaluation modules
Available modules
-Error
-MAE
-MSE
-RMSE
-MAPE
-DAC
-RAE
-RRSE
Loading module by name
MAE


In [4]:
# evaluate forecaster
print("Evaluate forecaster")
forecaster = WekaForecaster()
forecaster.fields_to_forecast = ["Adj Close"]
forecaster.base_forecaster = Classifier(classname="weka.classifiers.functions.LinearRegression")
forecaster.tslag_maker.timestamp_field = "Date"
forecaster.tslag_maker.adjust_for_variance = False

# add overlay data
forecaster.overlay_fields="Open,High,Low,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD"
print(forecaster.is_using_overlay_data)
print("forecast with overlays")
print(forecaster.forecast_with_overlays)

forecaster.tslag_maker.max_lag=7
forecaster.tslag_maker.min_lag=1
forecaster.tslag_maker.periodicity= Periodicity(periodicity="DAILY")



# print(help(forecaster.tslag_maker.periodicity))
# periodicity=None

forecaster.tslag_maker.include_powers_of_time = True
forecaster.tslag_maker.include_timelag_products = True
forecaster.tslag_maker.remove_leading_instances_with_unknown_lag_values = True

# forecaster.tslag_maker.add_month_of_year = True
# forecaster.tslag_maker.add_quarter_of_year = True

forecaster.tslag_maker.add_day_of_week=True
forecaster.tslag_maker.add_weekend_indicator=True

print("algorithm name: " + str(forecaster.algorithm_name))
print("command-line: " + forecaster.to_commandline())
print("lag maker: " + forecaster.tslag_maker.to_commandline())

evaluation = TSEvaluation(airline_data, 0.1)
evaluation.evaluate_on_training_data = True
evaluation.evaluate_on_test_data = True
# evaluation.prime_window_size = forecaster.tslag_maker.max_lag
evaluation.prime_for_test_data_with_test_data = True
evaluation.rebuild_model_after_each_test_forecast_step = False
evaluation.forecast_future = True
evaluation.horizon = 1
evaluation.prime_window_size=7
evaluation.evaluation_modules = "MAE,RMSE"


evaluation.evaluate(forecaster)
print(evaluation)
if evaluation.evaluate_on_training_data or evaluation.evaluate_on_test_data:
    print(evaluation.summary())
# if evaluation.evaluate_on_training_data:
#     print("Predictions (training data): " + evaluation.predictions_for_training_data(1).summary)
# if evaluation.evaluate_on_test_data:
#     print("Predictions (test data): " + evaluation.predictions_for_test_data(1).summary)
#     preds = evaluation.predictions_for_test_data(1)
#     print("Counts for targets: " + str(preds.counts_for_targets()))
#     print("Errors for target 'Adj Close': " + str(preds.errors_for_target("Adj Close")))
#     print("Errors for all targets: " + str(preds.predictions_for_all_targets()))
# if evaluation.training_data is not None:
#     print("Future forecasts (training)\n" + evaluation.print_future_forecast_on_training_data(forecaster))
# if evaluation.test_data is not None:
#     print("Future forecasts (test)\n" + evaluation.print_future_forecast_on_test_data(forecaster))

# if evaluation.evaluate_on_training_data:
#     print(evaluation.print_predictions_for_training_data("Predictions (training)", "Adj Close", 1))

# if evaluation.evaluate_on_test_data:
#     print(evaluation.print_predictions_for_test_data("Predictions (test)", "Adj Close", 1))

Evaluate forecaster
True
forecast with overlays
<bound method OverlayForecaster.forecast_with_overlays of Forecaster has not been built yet!>
algorithm name: LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4
command-line: weka.classifiers.timeseries.WekaForecaster -F "Adj Close" -O Open,High,Low,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD -trim-leading -L 1 -M 7 -B 2 -C 2 -G Date -dayofweek -weekend -conf 0 -P 0.95 -W "weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4"
lag maker: weka.filters.supervised.attribute.TSLagMaker -F "Adj Close" -O Open,High,Low,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD -trim-leading -L 1 -M 7 -G Date -dayofweek -weekend
=== Evaluation setup ===

Relation: BTC-2y
# Training instances: 634
# Test instances: 70
Evaluate on training data: True
Evaluate on test data: True
Horizon: 1
Prime window size: 7
Prime for test data with test data: True
Rebuild model after each test fo

In [5]:
# build forecaster
print("Build/use forecaster")
airline_train, airline_test = airline_data.train_test_split(90.0)
forecaster = WekaForecaster()
forecaster.fields_to_forecast = ["Adj Close"]
forecaster.base_forecaster = Classifier(classname="weka.classifiers.functions.LinearRegression")
forecaster.fields_to_forecast = "Adj Close"
forecaster.build_forecaster(airline_train)
num_prime_instances = 10
airline_prime = Instances.copy_instances(airline_train, airline_train.num_instances - num_prime_instances, num_prime_instances)
forecaster.prime_forecaster(airline_prime)
num_future_forecasts = airline_test.num_instances
preds = forecaster.forecast(num_future_forecasts)
print("Actual,Predicted,Error")
for i in range(num_future_forecasts):
    actual = airline_test.get_instance(i).get_value(0)
    predicted = preds[i][0].predicted
    error = actual - predicted
    print("%f,%f,%f" % (actual, predicted, error))

Build/use forecaster
Actual,Predicted,Error
1681660800000.000000,0.284440,1681660799999.715576
1681747200000.000000,0.284061,1681747199999.715820
1681833600000.000000,0.289436,1681833599999.710449
1681920000000.000000,0.296046,1681919999999.703857
1682006400000.000000,0.298370,1682006399999.701660
1682092800000.000000,0.302348,1682092799999.697754
1682179200000.000000,0.307351,1682179199999.692627
1682265600000.000000,0.310066,1682265599999.689941
1682352000000.000000,0.313132,1682351999999.686768
1682438400000.000000,0.318076,1682438399999.681885
1682524800000.000000,0.320824,1682524799999.679199
1682611200000.000000,0.324092,1682611199999.676025
1682697600000.000000,0.328801,1682697599999.671143
1682784000000.000000,0.332264,1682783999999.667725
1682870400000.000000,0.335690,1682870399999.664307
1682956800000.000000,0.340120,1682956799999.659912
1683043200000.000000,0.343870,1683043199999.656250
1683129600000.000000,0.347250,1683129599999.652832
1683216000000.000000,0.351605,16832159

In [6]:
# serialization (if supported)
print("Serialization")
model_file = "base.model"
if forecaster.base_model_has_serializer:
    forecaster.save_base_model(model_file)
    forecaster2 = WekaForecaster()
    forecaster2.load_base_model(model_file)
    print(forecaster2.to_commandline())
else:
    print("Base model has no serializer, falling back to generic serialization")
    serialization_write(model_file, forecaster.base_forecaster)
    cls = Classifier(jobject=serialization_read(model_file))
    print(cls.to_commandline())

Serialization
Base model has no serializer, falling back to generic serialization
weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4


In [7]:
# state management
print("State")
model_file = "state.ser"
if forecaster.uses_state:
    forecaster.serialize_state(model_file)
    forecaster2 = WekaForecaster()
    forecaster2.load_serialized_state(model_file)
    print(forecaster2.to_commandline())
else:
    print("Forecaster does not use state, falling back to generic serialization")
    serialization_write(model_file, forecaster)
    forecaster2 = WekaForecaster(jobject=serialization_read(model_file))
    print(forecaster2.to_commandline())

State
Forecaster does not use state, falling back to generic serialization
weka.classifiers.timeseries.WekaForecaster -F "Adj Close" -L 1 -M 12 -B 2 -C 2 -G ArtificialTimeIndex -conf 0 -P 0.95 -W "weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4"


In [ ]:
jvm.stop()